# Definition of Excess

This notebook illustrates an inconsistency in the definition of excess between forward and backwards folding. Note this is not a real issue, but just related to the way we currently define excess in the forward folding case.

In [13]:
from gammapy.stats import WStatCountsStatistic
from gammapy.datasets import SpectrumDatasetOnOff, SpectrumDataset
from gammapy.maps import RegionGeom, MapAxis
from gammapy.modeling.models import SkyModel, PowerLawSpectralModel
from astropy import units as u
import numpy as np

In [14]:
n_on = 100
n_off = 20
alpha = 1

## Counts Statistic Case

We first start with counts statistics case:

In [15]:
wstat = WStatCountsStatistic(
    n_on=n_on, n_off=n_off, alpha=alpha
)

In [16]:
print(f"Counts  : {wstat.n_on}")
print(f"Bkg.    : {wstat.n_bkg}")
print(f"Sqrt(TS): {wstat.sqrt_ts:.2f}")
print(f"Excess  : {wstat.n_sig:.2f}")

Counts  : 100
Bkg.    : 20
Sqrt(TS): 7.63
Excess  : 80.00


Excess and background add up to the total counts in this case. However, and this is important to keep in mind, the excess is defined as the **excess of counts above the background in the test hypothesis case**. So strictly speaking it is only valid once one adopts the test hypothesis. 

## On-Off Dataset Case

No we repeat the same scenario with the dataset case:

In [17]:
axis = MapAxis.from_energy_edges([1, 10] * u.TeV)
geom = RegionGeom.create(region=None, axes=[axis])
d = SpectrumDatasetOnOff.create(geom=geom)

d.counts += 100
d.counts_off += 20
d.acceptance += 1
d.acceptance_off += 1
d.mask.data += True
d.exposure += 1e10 * u.m ** 2 * u.s

### Current definition

In [18]:
# The amplitude chosen such that npred = counts
pwl = PowerLawSpectralModel(amplitude=8/9 * u.Unit("1e-12 cm-2 s-1 TeV-1"))
d.models = SkyModel(spectral_model=pwl, name="test-source")

stat = d.stat_sum()
npred = d.npred()

d.models["test-source"].spectral_model.amplitude.value = 0

stat_null = d.stat_sum()
npred_null = d.npred()

sqrt_ts = float(np.sqrt(stat_null - stat))
npred_excess = float((npred - npred_null).data)

In [19]:
print(f"Counts  : {int(d.counts.data)}")
print(f"Bkg.    : {int(d.background.data)}")
print(f"Sqrt(TS): {sqrt_ts:.2f}")
print(f"Excess  : {npred_excess:.2f}")

Counts  : 100
Bkg.    : 20
Sqrt(TS): 7.63
Excess  : 40.00


This case shows the same signficance but a different excess definition. In this case the **excess is defined as the difference in predicted counts between the test hypothesis and the null hypothesis**. Which is different from the definiton above. In the case of `wstat` the background is reoptimized for the null hypothesis and the background level is different for the test and null hypothesis case. This results in the different number for the excess. From this we can conclude a modified consistent definition.

### Modified consistent definiton:

In [20]:
# The amplitude chosen such that npred = counts
pwl = PowerLawSpectralModel(amplitude=8/9 * u.Unit("1e-12 cm-2 s-1 TeV-1"))
d.models = SkyModel(spectral_model=pwl, name="test-source")

stat = d.stat_sum()
npred = d.npred()

# one subtracts the counts from the test source in the test hypothesis case
npred_null = npred - d.npred_signal("test-source")

d.models["test-source"].spectral_model.amplitude.value = 0

stat_null = d.stat_sum()

sqrt_ts = float(np.sqrt(stat_null - stat))
npred_excess = float((npred - npred_null).data)

In [21]:
print(f"Counts  : {int(d.counts.data)}")
print(f"Bkg.    : {int(d.background.data)}")
print(f"Sqrt(TS): {sqrt_ts:.2f}")
print(f"Excess  : {npred_excess:.2f}")

Counts  : 100
Bkg.    : 20
Sqrt(TS): 7.63
Excess  : 80.00


However this means that the current quantity we store `npred_null` (total predicted counts in the null hypothesis case) is not relevant in the excess definiton. The question which how we denote the quantity `npred - npred_source` as in the modified consistent definiton.